# Add message history (memory) （添加消息记录基于内存）
RunnableWithMessageHistory允许我们向某些类型的链中添加消息历史记录。
具体来说，它可用于任何接受以下之一作为输入的Runnable：
- BaseMessage序列
- 带有接受BaseMessage序列的键的字典
- 带有以字符串或BaseMessage序列形式接收最新消息的键和单独接收历史消息的键的字典

并将以下之一作为输出返回：
- 可以视为AIMessage内容的字符串
- BaseMessage序列
- 包含BaseMessage序列的键值对字典
  
让我们看一些示例来了解它是如何工作的。

## Example: Dict input, message output(字典输入,消息输出)

让我们创建一个简单的链，它以字典作为输入并返回一个BaseMessage。
在这种情况下，输入中的“question”键代表我们的输入消息，“history”键是我们历史消息将被注入的地方。

In [44]:
# 我们将使用 Redis 来存储聊天消息历史记录(pip install redis)

from typing import Optional
from langchain.chat_models import ChatOpenAI
from langchain.memory.chat_message_histories import RedisChatMessageHistory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

model = ChatOpenAI(
  openai_api_key="",
  openai_api_base="",
)

promt = ChatPromptTemplate.from_messages(
      [
        ("system", "你是一个擅长 {ability} 的助手"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = promt | model




### 添加历史消息

为了将消息历史记录添加到我们的原始链中，我们将其包装在RunnableWithMessageHistory类中。
至关重要的是，我们还需要定义一个方法，该方法接受一个session_id字符串，并根据它返回BaseChatMessageHistory。对于相同的输入，这个方法应该返回等效的输出。
在这种情况下，我们还希望指定input_messages_key（要视为最新输入消息的键）和history_messages_key（要向其中添加历史消息的键）。


In [45]:
chain_with_history = RunnableWithMessageHistory(
  chain,
  lambda session_id: RedisChatMessageHistory(session_id,url=""),
  input_messages_key="question",
  history_messages_key="history"
)

### 使用配置调用
每当我们使用消息历史记录调用我们的链时，需要包含一个包含session_id的配置。

In [46]:
config={"configurable": {"session_id": "<SESSION_ID>"}}

在相同的配置下，我们的链应该从相同的聊天消息历史中拉取。

In [51]:
chain_with_history.invoke(
    {"ability": "math", "question": "什么是余弦?"},
    config={"configurable": {"session_id": "foobar"}},
)

AIMessage(content='余弦是三角函数中的一种，表示为cos。在直角三角形中，余弦是指一个角的邻边与斜边的比值。在数学中，余弦函数通常用来描述角度和直角三角形之间的关系，它可以帮助我们计算角度和边长的关系。余弦函数的定义域为实数集，值域为[-1, 1]。')

In [52]:
chain_with_history.invoke(
    {"ability": "math", "question": "那么它反函数是什么?"},
    config={"configurable": {"session_id": "foobar"}},
)

AIMessage(content='余弦函数的反函数被称为反余弦函数，通常表示为arccos或者cos^{-1}。反余弦函数的定义域为[-1, 1]，值域为[0, π]。它的作用是将一个给定的余弦值转换成对应的角度，因此可以用来求解角度。在三角函数中，反余弦函数是三角函数中的反函数之一，可以帮助我们从已知的余弦值中求解对应的角度。')

## Example: messages input, dict output （消息输入，字典输出）

In [53]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel({"output_message":model})
chain_with_history = RunnableWithMessageHistory(
  chain,
  lambda session_id: RedisChatMessageHistory(session_id,url=""),
  output_messages_key="output_message",
)

chain_with_history.invoke(
  [HumanMessage(content="Simone de Beauvoir 认为自由是什么?")],
  config={"configurable": {"session_id": "baz"}},
)

{'output_message': AIMessage(content='Simone de Beauvoir认为自由是人类最基本的权利和价值观，是人类的本质特征。她认为自由是指个体在不受外部压迫和限制的情况下，能够自主地做出选择和行动的能力。自由也包括了对自己的生活和命运负责任的能力。在她的著作中，她强调了自由与责任、选择与决定的密切关系，认为只有在自由的基础上才能实现个体的真正意义和价值。')}

In [54]:
chain_with_history.invoke(
  [HumanMessage(content="这与Sartre相比如何？")],
    config={"configurable": {"session_id": "baz"}},
)

{'output_message': AIMessage(content='西蒙·德·波伏儂和让-保罗·萨特是20世纪法国哲学家中的两位重要人物，他们的思想有很多相似之处，但也有一些不同之处。\n\n在自由的理念上，两人都认为自由是人类的本质特征，是人类存在的基础。然而，在对自由的理解上，两人有一些不同。波伏儂强调了自由与责任的关系，认为自由的行使需要伴随着责任和选择，个体应该对自己的行为和选择承担责任。而萨特则更强调了自由的绝对性和孤立性，他认为人类是绝对自由的存在，个体要为自己的一切选择和行为负责，没有任何外在的道德准则和指导。\n\n总的来说，波伏儂更加强调了自由与责任的内在联系，而萨特更加强调了自由的绝对性和个体的孤立性。两人对自由的理解有一些差异，但都对人类自由的重要性和本质特征有着深刻的思考。')}